# Analysis of Neighborhoods in Toronto, Canada
## Part 2 - Dataframe with Geodata

Marnilo C

In [2]:
# Import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# for easier handling or URLs
import requests

# for parsing HTML
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [18]:
# Create the dataframe df1 from HTML table data at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

# Scrape the data
url1 = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url1).text

soup = BeautifulSoup(html_data,'lxml')
#print(soup.prettify())

html_table = soup.find('table',{'class':'wikitable sortable'})
#html_table_data

# Create the dataframe.
def parse_html_table(table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

df1 = parse_html_table(html_table)

# Clean the dataframe
df1.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood\n': 'Neighbourhood'}, inplace=True)
df1.drop(df1.loc[df1.Borough == 'Not assigned'].index, inplace=True)
df1['Neighbourhood'] = df1['Neighbourhood'].str.strip()
df1.loc[df1['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df1['Borough']
df1 = df1.groupby('PostalCode').agg({'Borough':'first',
                               'Neighbourhood': ', '.join}).reset_index()

print('Dataframe df1 created and loaded with data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.')
print('Dataframe df1 shape:', df1.shape)
df1.head()

Dataframe df1 created and loaded with data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.
Dataframe df1 shape: (103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
# Download the CSV data
!wget -q -O 'toronto_geodata.csv' http://cocl.us/Geospatial_data
print('CSV data downloaded.')

CSV data downloaded.


In [23]:
# Create dataframe from the CSV file at http://cocl.us/Geospatial_data
df2 = pd.read_csv('toronto_geodata.csv', header=0)
df2.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

print('Dataframe df2 created and loaded with data from http://cocl.us/Geospatial_data')
print('Dataframe df2 shape:', df2.shape)
df2.head()

Dataframe df2 created and loaded with data from http://cocl.us/Geospatial_data
Dataframe df2 shape: (103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
# Create new dataframe with boroughs, neighborhoods, and geodata
df3 = pd.merge(df1, df2, on='PostalCode', how='inner')
print('Dataframe df3 created by inner join of df1 and df2.')
print('Dataframe df3 shape:', df3.shape)
df3.head()

Dataframe df3 created by inner join of df1 and df2.
Dataframe df3 shape: (103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
